In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

In [ ]:
NUM_LANES = 2
INPUT_SIZE = (288, 800)
CLS_SHAPE = (20, 100, NUM_LANES)

INPUT_SHAPE = (*INPUT_SIZE, 3)
OUTPUT_SHAPE = CLS_SHAPE

PREFETCH_SIZE = 200

DTYPE = tf.float32

TAKE_LLAMAS_MAX = 1000
TAKE_CULANE_MAX = 0
TAKE_MIWULA_MAX = 0 # all of them

# Ratio of validation data vs training data
VALID_RATIO = 0.3

BATCH_SIZE = 8 # tune to available GPU memory
PREFETCH_SIZE = 200 # tune to available memory


EPOCHS = 20 # training epochs


In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))
    
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
import json
import os
from unsupervised_llamas.label_scripts.spline_creator import get_horizontal_values_for_four_lanes

BASE_PATH = '/Volumes/tank_data/Archiv/ML/llamas/labels/'

LLAMAS_SHAPE = (717, 1276, 3)
#MASK_SHAPE = (712, 1272, 3)

IMAGE_SHAPE = INPUT_SHAPE
MASK_SHAPE = OUTPUT_SHAPE

PARALLEL_JOBS = 4# tf.data.experimental.AUTOTUNE


def resize_img(img, shape):
    w = img.shape[1]
    h = img.shape[0]
    
    ratio = shape[1] / shape[0]
    
    tgt = img[h-int(w/ratio):h,0:w]
    tgt = cv2.resize(tgt, (shape[1], shape[0]))
    return tgt

def draw_spline(img, points, color):
    n = np.array(points)
    idx = np.arange(0, len(n)).reshape(-1,1)
    idx = idx[n >= 0]
    n = n[n >= 0]
    n = n.reshape(-1,1)
    pts = np.hstack((n, idx))
    
    cv2.polylines(img, np.int32([pts]), isClosed=False, color=color, thickness=10, lineType=cv2.LINE_8)
    
    del n
    del idx
    del pts

def generate_mask(file):
    file = file.numpy().decode("utf-8")
    mask = np.zeros(LLAMAS_SHAPE, dtype='uint8')
    
    lines = get_horizontal_values_for_four_lanes(file)
    l0 = lines[1]
    r0 = lines[2]
    
    draw_spline(mask, l0, (0,255,0))
    draw_spline(mask, r0, (255,0,0))
    
    mask = resize_img(mask, MASK_SHAPE)
    return mask[:,:,0:2]

def read_image(file):
    file = file.numpy().decode("utf-8")
    with open(file, 'r') as fp:
        meta = json.load(fp)
        
    image_path = os.path.dirname(file)
    image_path = image_path.replace('/labels/', '/color_images/')
    img_name = meta['image_name'] + '_color_rect.png'
    fname = os.path.join(image_path, img_name)
    
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return resize_img(img, IMAGE_SHAPE)

def process_json(json):
    img = tf.py_function(read_image, [json], Tout=DTYPE)
    mask = tf.py_function(generate_mask, [json], Tout=DTYPE)
    
    img = tf.reshape(img, shape=IMAGE_SHAPE)
    mask = tf.reshape(mask, shape=MASK_SHAPE)
    return img, mask

SHUFFLE_SIZE = 100000

llamas_train_ds = tf.data.Dataset.list_files(os.path.join(BASE_PATH, 'train', '*/*.json'))
llamas_train_ds = llamas_train_ds.shuffle(SHUFFLE_SIZE)

llamas_train_ds = llamas_train_ds.take(TAKE_LLAMAS_MAX)
LLAMAS_TRAIN_LEN = len(list(llamas_train_ds))
print('LLAMAS training images: %d' % LLAMAS_TRAIN_LEN)

llamas_train_ds = llamas_train_ds.map(process_json, num_parallel_calls=PARALLEL_JOBS)

llamas_valid_ds = tf.data.Dataset.list_files(os.path.join(BASE_PATH, 'valid', '*/*.json'))
llamas_valid_ds = llamas_valid_ds.shuffle(SHUFFLE_SIZE)

llamas_valid_ds = llamas_valid_ds.take(TAKE_LLAMAS_MAX)
LLAMAS_VALID_LEN = len(list(llamas_valid_ds))
print('LLAMAS validation images: %d' % LLAMAS_VALID_LEN)

llamas_valid_ds = llamas_valid_ds.map(process_json, num_parallel_calls=PARALLEL_JOBS)

for img, mask in llamas_valid_ds.take(3):
    m = tf.zeros((20,100,1), dtype=DTYPE)
    m = tf.concat([mask, m], -1)
    display([img, m])


In [ ]:
import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter('ultrafast_quant_edgetpu.tflite', experimental_delegates=[tflite.load_delegate('libedgetpu.1.dylib')])
interpreter.allocate_tensors()

def set_input(interpreter, tensor):
    input_details = interpreter.get_input_details()[0]
    #print(input_details)
    tensor_index = input_details['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    
    scale, zero_point = input_details['quantization']
    
    input_tensor[:, :] = tensor
    
def run_ultrafast(interpreter, inp_img):
    set_input(interpreter, inp_img)
    
    #s = time.time()
    interpreter.invoke()
    #e = time.time()
    #print(e-s)
    
    output_details = interpreter.get_output_details()[0]
    #print(output_details)
    output = interpreter.get_tensor(output_details['index'])
    # Outputs from the TFLite model are uint8, so we dequantize the results:
    #scale, zero_point = output_details['quantization']
    #output = scale * (output - zero_point)
    
    return output

In [ ]:
llamas_valid_ds = llamas_valid_ds.prefetch(100).cache()

for img, mask in llamas_valid_ds.take(50):
    output = run_ultrafast(interpreter, img)
    
    m = tf.zeros((20,100,1), dtype=DTYPE)
    m = tf.concat([output[0], m], -1)
    
    m2 = tf.zeros((20,100,1), dtype=DTYPE)
    m2 = tf.concat([mask, m2], -1)
    
    display([img, m2, m])
    
    #print(output)

In [ ]:
def resize_img(img, shape):
    w = img.shape[1]
    h = img.shape[0]
    
    ratio = shape[1] / shape[0]
    
    tgt = img[h-int(w/ratio):h,0:w]
    tgt = cv2.resize(tgt, (shape[1], shape[0]))
    return tgt


def pipeline(img):
    img = resize_img(img, INPUT_SHAPE)
    
    out = run_ultrafast(interpreter, img)
    m = tf.zeros((20,100,1), dtype=DTYPE)
    m = tf.concat([out[0], m], -1)
    
    #print(m.shape)
    #out_img = (m.numpy() * 255).astype(np.int)
    #print(out_img)
    ratio = m.shape[0] / m.shape[1]
    
    
    new_w = img.shape[1]
    new_h = int(new_w * ratio)
    
    rescale = cv2.resize(m.numpy(), (new_w, new_h))
    
    min_val = np.min(rescale)
    max_val = np.max(rescale)
    rescale = ((rescale - min_val) / (max_val - min_val)) * 255
    
    tmp = np.zeros(img.shape, dtype=np.uint8)
    tmp[128:288,:] = rescale
    
    overlay = cv2.addWeighted(img, 1, tmp, 1, 0)
    
    return overlay

In [ ]:
test = cv2.imread('test.png')
test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)


out = pipeline(test)

display([test, out])

out.shape

In [ ]:
from moviepy.editor import VideoFileClip

clip = 'out2.mp4'

clip1 = VideoFileClip(clip)
driving_clip = clip1.fl_image(pipeline)
driving_clip.write_videofile('out2_ultrafast.mp4', audio=False)